In [1]:
import pandas as pd
import os

# -----------------------------
# 0. 파일 경로 설정
# -----------------------------
cohort_path = "./cohort/cohort_ver43_with_ecg.csv"     # 입력: ver43
ed_path      = "../../data/mimic-iv-ed/ed/edstays.csv"    # MIMIC-IV-ED edstays (경로는 환경에 맞게 수정)
output_path  = "./cohort/cohort_ver44_with_ed.csv"     # 출력: ver44


# -----------------------------
# 1. Cohort(ver43) 로드
# -----------------------------
print(f"Loading cohort: {cohort_path}")
cohort = pd.read_csv(cohort_path)

# subject_id / hadm_id가 있는지 확인 (없으면 에러)
required_cols = ["subject_id", "hadm_id"]
missing = [c for c in required_cols if c not in cohort.columns]
if missing:
    raise ValueError(f"Cohort에 필요한 컬럼이 없습니다: {missing}")


# -----------------------------
# 2. ED edstays 로드
# -----------------------------
print(f"Loading ED stays: {ed_path}")
ed = pd.read_csv(ed_path)

# 기본적으로 MIMIC-IV-ED edstays는 다음 컬럼들을 가진다고 가정:
# subject_id, hadm_id, stay_id, intime, outtime, ...
for col in ["subject_id", "hadm_id", "intime", "outtime"]:
    if col not in ed.columns:
        raise ValueError(f"ED edstays에 {col} 컬럼이 없습니다. 실제 스키마를 확인해 주세요.")


# -----------------------------
# 3. ED 시각 컬럼 datetime 변환
# -----------------------------
ed["intime"] = pd.to_datetime(ed["intime"])
ed["outtime"] = pd.to_datetime(ed["outtime"])


# -----------------------------
# 4. 입실 시간 기준으로 하나의 ED stay만 선택
#    (동일 subject_id+hadm_id 에 ED stay가 여러 개일 수 있으므로
#     가장 이른 intime 을 대표 ED stay로 사용)
# -----------------------------
ed_sorted = ed.sort_values(["subject_id", "hadm_id", "intime"])
ed_dedup = ed_sorted.drop_duplicates(subset=["subject_id", "hadm_id"], keep="first").copy()

# -----------------------------
# 5. ED LOS 계산 (hour 단위)
#    ed_los = outtime - intime
# -----------------------------
ed_dedup["ed_intime"] = ed_dedup["intime"]
ed_dedup["ed_outtime"] = ed_dedup["outtime"]

# Timedelta → 시간(hour) 단위 float
ed_dedup["ed_los"] = (ed_dedup["ed_outtime"] - ed_dedup["ed_intime"]).dt.total_seconds() / 3600.0


# -----------------------------
# 6. ED 체류 식별자 (ed_stay_id)
#    - 기본: stay_id 사용
#    - 예외: stay_id가 없고 hadm_id만 있는 경우 hadm_id로 대체
# -----------------------------
if "stay_id" in ed_dedup.columns:
    ed_dedup["ed_stay_id"] = ed_dedup["stay_id"]
else:
    # stay_id가 아예 없으면 hadm_id를 그대로 사용
    ed_dedup["ed_stay_id"] = ed_dedup["hadm_id"]

# stay_id가 일부 NaN인 경우도 hadm_id로 채우기
ed_dedup["ed_stay_id"] = ed_dedup["ed_stay_id"].fillna(ed_dedup["hadm_id"])


# -----------------------------
# 7. Cohort와 ED 정보 merge
#    기준: subject_id + hadm_id (left join)
# -----------------------------
merge_cols = [
    "subject_id",
    "hadm_id",
    "ed_intime",
    "ed_outtime",
    "ed_los",
    "ed_stay_id",
]

ed_for_merge = ed_dedup[merge_cols].copy()

print("Merging cohort with ED stays...")
cohort_merged = cohort.merge(
    ed_for_merge,
    on=["subject_id", "hadm_id"],
    how="left",
    validate="m:1"   # 한 (subject, hadm)에 하나의 ED stay만 매칭되도록 기대
)


# -----------------------------
# 8. 저장
# -----------------------------
os.makedirs(os.path.dirname(output_path), exist_ok=True)
cohort_merged.to_csv(output_path, index=False)
print(f"Saved cohort with ED columns to: {output_path}")

# 간단한 요약 출력
print("\n[Summary]")
print("cohort rows:", len(cohort))
print("cohort + ED rows:", len(cohort_merged))
print("ED 정보가 존재하는 row 수:", cohort_merged["ed_intime"].notna().sum())


Loading cohort: ./cohort/cohort_ver43_with_ecg.csv
Loading ED stays: ../../data/mimic-iv-ed/ed/edstays.csv
Merging cohort with ED stays...
Saved cohort with ED columns to: ./cohort/cohort_ver44_with_ed.csv

[Summary]
cohort rows: 1930
cohort + ED rows: 1930
ED 정보가 존재하는 row 수: 1930
